In [1]:
import psycopg2
import sched, time
import numpy as np
import sqlalchemy

In [ ]:
# To connect to a Postgres DB

conn = psycopg2.connect(
    host="your_host",
    port="your_port",
    database="your_database",
    user="your_username",
    password="your_password"
)

cursor = conn.cursor()

In [ ]:
"""
# Read the SQL file
with open('C:\\Users\\valer\\Desktop\\Advanced_Databases\\', 'r') as file:
    sql_content = file.read()

# Split the SQL content into separate queries
sql_queries = sql_content.split(';')

# Remove empty strings or whitespace-only queries
sql_queries = [query.strip() for query in sql_queries if query.strip()]
"""

In [ ]:
# Open the CSV file and read its contents
with open('rides.csv', 'r') as file:
    reader = csv.reader(file)
    rides_data = list(reader)

# repeat for the others...

In [ ]:
def query_executer(i):
    print("Executing function...")  # Replace this with the function's actions
    prob_list = np.random.randint(0, 1, 5)
    binary_prob = []

    for x in prob_list[0:2]: 
        if x < (1/30): # to check (queries that run on avg every 30 secs)
            binary_prob.append(1)
        else:
            binary_prob.append(0)
    for x in prob_list[2:6]:
        if x < (1/900): # to check (queries that run on avg every 15 min)
            binary_prob.append(1)
        else:
            binary_prob.append(0)

    # Query 1
    if binary_prob[0] == 1:
        random_status = np.random.randint(1,3)
        random_driver = np.random.randint(1,150) # to check
        query_to_insert = sqlalchemy.text(f"""
        UPDATE driver
        SET driver_status = '{random_status}'
        WHERE driver_id = '{random_driver}';
        """)
        db_conn.execute(query_to_insert)

    # Query 2 to 5
    def insert_fun(dataset, table_name):
        random_row = random.choice(dataset)
        query_to_insert = sqlalchemy.text(f"""
        INSERT INTO {table_name} VALUES {tuple(random_row)}
        """)
        db_conn.execute(query_to_insert)
    
    if binary_prob[1] == 1:
            table_name = 'rides'
            insert_fun(table_name, data_rides)
    
    if binary_prob[2] == 1:
            table_name = 'user'
            insert_fun(table_name, data_user)
            
    if binary_prob[3] == 1:
            table_name = 'driver'
            insert_fun(table_name, data_driver)
    
    if binary_prob[4] == 1:
            table_name = 'vehicle'
            insert_fun(table_name, data_vehicle)
    
    if i%900 == 0:
        # Query 6
        query_to_insert = sqlalchemy.text(f"""
        SELECT COUNT(*)
        FROM rides
        WHERE dropoff_date >= DATE_SUB(NOW(), INTERVAL 15 MINUTE);
        """)
        
        # Query 7
        query_to_insert = sqlalchemy.text(f"""
        SELECT COUNT(*)
        FROM driver
        WHERE driver_status = '0';
        """)
        
        # Query 8
        query_to_insert = sqlalchemy.text(f"""
        DELETE FROM rides
        WHERE dropoff_date < DATEADD(day, -1, GETDATE());
        """)
        db_conn.execute(query_to_insert)
    i+=1
  

        



In [ ]:
# Create a scheduler
scheduler = sched.scheduler(time.time, time.sleep)

# Define a function to repeatedly execute 'my_function' every 30 seconds
def repeat_function(scheduler, interval, action, total_runtime):
    if total_runtime < 30:  # Stop condition after 300 seconds (5 minutes)
        scheduler.enter(interval, 1, repeat_function, (scheduler, interval, action, total_runtime + interval))
        action()
    else:
        print("Stopped function after 300 seconds")

# Schedule the function to run every 1 second

# !!!!! HOW TO GIVE INPUT TO query_executer (I.E. i)
repeat_function(scheduler, 1, query_executer, 0)

# Run the scheduler
scheduler.run()